In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import keras
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
import tensorflow_datasets as tfds
from keras_preprocessing.text import text_to_word_sequence
from sklearn.model_selection import train_test_split

2023-09-13 11:20:15.342539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 11:20:15.421633: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 11:20:15.424746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-13 11:20:15.424755: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
data_processed = pd.read_csv("/home/luades/code/irismarechal1997/moder_ia/data/processed_dataset_v1.csv")

# if processed :
X = data_processed["text"]
y = data_processed["offensive"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
# else : 
#     X = X_proc
#     y = data_processed["offensive"]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

In [3]:
y_train

75225    1
18048    1
16710    1
32625    0
87227    1
        ..
48056    1
59011    0
77049    0
67224    0
71530    1
Name: offensive, Length: 77533, dtype: int64

In [4]:
### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

print(f'There are {vocab_size} different words in your corpus')

There are 58592 different words in your corpus


In [5]:
# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

In [6]:
len(X_train_pad[0])

402

In [7]:
X_train_pad.shape, y_train.shape

((77533, 402), (77533,))

In [11]:
### Let's build the neural network now
from tensorflow.keras import layers, Sequential

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.LSTM(20, return_sequences=True, activation="tanh"))
model.add(layers.LSTM(20, activation="tanh"))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          2929650   
                                                                 
 lstm_2 (LSTM)               (None, None, 20)          5680      
                                                                 
 lstm_3 (LSTM)               (None, 20)                3280      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,938,631
Trainable params: 2,938,631
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          epochs=5, 
          batch_size=128,
          callbacks=[es]
         )

Epoch 1/5
606/606 [==============================] - 306s 505ms/step - loss: 0.4836 - accuracy: 0.7568
Epoch 2/5
606/606 [==============================] - 401s 662ms/step - loss: 0.4421 - accuracy: 0.7842
Epoch 3/5
606/606 [==============================] - 368s 608ms/step - loss: 0.4126 - accuracy: 0.8040
Epoch 4/5
606/606 [==============================] - 431s 712ms/step - loss: 0.3875 - accuracy: 0.8190
Epoch 5/5
606/606 [==============================] - 415s 684ms/step - loss: 0.3647 - accuracy: 0.8327


In [ ]:
model.evaluate(X_test_pad, y_test)[[1]]

In [ ]:
model.predict(scaler.transform(X_test))

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model2.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model2.add(layers.GRU(20, activation="tanh"))
model2.add(layers.Dense(1, activation="sigmoid"))
model2.summary()